## Setup MongoDB

In [1]:
import subprocess

# Define the command to run the Streamlit app
command = "streamlit run dashboard.py"

# Run the command in the background
process = subprocess.Popen(command, shell=True)


## Raccolta dati da WorldNewsApi

In [6]:
import worldnewsapi
from worldnewsapi.rest import ApiException
#newsapi_configuration = worldnewsapi.Configuration(api_key={'apiKey': '0f2d5e3a0af046d3b07a243c2bf5472b'})
newsapi_configuration = worldnewsapi.Configuration(api_key={'apiKey': 'b9ee5e19c8604b92a039d4a95c666e96'})


news_urls = [
    "https://www.ansa.it/",
#     "https://www.fanpage.it/",
#     "https://www.corriere.it/",
    "https://www.ilfattoquotidiano.it/",
#     "https://www.ilgiornale.it/",
    "https://www.ilsole24ore.com/",
    "https://www.repubblica.it/",
#     "https://www.lastampa.it/",
    "https://www.ilmattino.it/"
#   "https://www.lastampa.it/",
#     # "https://www.liberoquotidiano.it/",
#     "https://www.tgcom24.mediaset.it/"
]
# Converti la lista degli URL in una stringa separata da virgole
news_sources = ",".join(news_urls)

try:
	newsapi_instance = worldnewsapi.NewsApi(worldnewsapi.ApiClient(newsapi_configuration))

	max_results = 1000  # replace with your maximum
	offset = 0
	all_results = []

	while len(all_results) < max_results:

		request_count = min(100, max_results - len(all_results)) # request 100 or the remaining number of articles

		response = newsapi_instance.search_news(
			source_countries='it',
			language='it',
            news_sources= news_sources,
			earliest_publish_date=   '2023-06-10 19:48:05', 
			latest_publish_date='2023-06-20', 
			sort="publish-time",
			sort_direction="asc",
			#min_sentiment=-1,
			#max_sentiment=1,
			offset=offset,
			number=request_count)
		
		print("Retrieved " + str(len(response.news)) + " articles. Offset: " + str(offset) + "/" + str(max_results) +
			  ". Total available: " + str(response.available) + ".")

		all_results.extend(response.news)
		offset += 100

except worldnewsapi.ApiException as e:
	print("Exception when calling NewsApi->search_news: %s\n" % e)

Retrieved 100 articles. Offset: 0/1000. Total available: 2459.
Retrieved 100 articles. Offset: 100/1000. Total available: 2459.
Retrieved 100 articles. Offset: 200/1000. Total available: 2459.
Retrieved 100 articles. Offset: 300/1000. Total available: 2459.
Retrieved 100 articles. Offset: 400/1000. Total available: 2459.
Retrieved 100 articles. Offset: 500/1000. Total available: 2459.
Retrieved 100 articles. Offset: 600/1000. Total available: 2459.
Retrieved 100 articles. Offset: 700/1000. Total available: 2459.
Retrieved 100 articles. Offset: 800/1000. Total available: 2459.
Retrieved 100 articles. Offset: 900/1000. Total available: 2459.


In [7]:
all_results[-1].publish_date


'2023-06-14 01:00:00'

In [7]:
print(pippo)

<bound method SearchNewsResponse.to_dict of SearchNewsResponse(offset=0, number=1, available=99, news=[NewsArticle(id=175764819, title='Saldi, al via il 3 gennaio. Confcommercio: giro d’affari da 4,8 miliardi', text='Archiviato il Capodanno, è gà tempo di saldi invernali: mercoledì 3 gennaio si parte in Valle d’Aosta, venerdì 5 in tutte le altre regioni. Saranno 15,8 milioni le famiglie che si dedicheranno allo shopping scontato e ogni persona spenderà circa 137 euro, per un giro di affari di 4,8 miliardi di euro: sono questi, secondo le stime dell’Ufficio Studi Confcommercio, i numeri dei saldi invernali. Spiega il presidente nazionale di Federazione Moda Italia-Confcommercio , Giulio Felloni: “Sono stime che evidenziano una tenuta della propensione al consumo degli italiani, dopo un anno complesso in cui la moda ha contribuito in maniera determinante alla discesa ed al contenimento dell’inflazione. Questi saldi rappresentano un’eccezionale opportunità per i consumatori che potranno t

## Elaborazione JSON ricevuto

In [8]:
# Initialize an empty list to store dictionaries
news_dicts = []

# Iterate over each NewsArticle object in all_results and convert it to a dictionary
for news_article in all_results:
    news_dicts.append(news_article.to_dict())

### Preprocessing dati

In [9]:
# Define the keys to exclude
keys_to_exclude = {"id", "image"}

# Initialize an empty list to store filtered dictionaries
filtered_news_dicts = []

# Iterate over each NewsArticle object in all_results and convert it to a filtered dictionary
for news_article in news_dicts:
    filtered_article = {k: v for k, v in news_article.items() if k not in keys_to_exclude}
    filtered_news_dicts.append(filtered_article)

### Scrittura csv

In [10]:
import csv
import re
import os

def clean_html_tags(text):
    """Remove HTML tags from the given text."""
    if text:
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove NBSP characters
        text = text.replace('\xa0', ' ').replace('&nbsp;', ' ')
        return text
    return text

# Ensure news_dicts is not empty
if filtered_news_dicts:
    # Retrieve field names from all elements of news_dicts
    field_names = set(field for news_article in filtered_news_dicts for field in news_article.keys())

    # Define the file name for the CSV
    csv_file = "news_data_filtered.csv"

    # Check if the file already exists
    file_exists = os.path.isfile(csv_file)

    # Open the file in append mode
    with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=sorted(field_names), extrasaction='ignore')

        # Write the header row only if the file does not already exist
        if not file_exists:
            writer.writeheader()

        # Write the data from news_dicts to the CSV file
        for news_article in filtered_news_dicts:
            # Clean the "summary" field if it exists
            if 'summary' in news_article:
                news_article['summary'] = clean_html_tags(news_article['summary'])
            
            try:
                writer.writerow(news_article)
            except ValueError as e:
                # Handle missing fields by filling with 'NaN'
                row = {field: news_article.get(field, 'NaN') for field in field_names}
                writer.writerow(row)

    print("CSV file has been successfully updated:", csv_file)
else:
    print("Error: filtered_news_dicts is empty. No data to write to CSV.")


CSV file has been successfully updated: news_data_filtered.csv
